In [1]:
import pandas as pd
from pandas import Series, DataFrame
import datetime
pd.set_option('display.max_columns',None) ##显示所有列
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, Float, Integer

In [6]:
def get_data_from_view(user,password,db_ip,db_name,port,view_name,dropdate):
    '''
    func:获取视图数据，并进行合并处理
    :param: user,password,db_ip,db_name,port,view_name数据库参数
    :param: dropdate 日期索引
    :return: all_data_df
    '''
    #连接数据库
    engine = create_engine('mysql+pymysql://' + user + ":" + password + "@" + db_ip + ":" + str(port) + "/" + db_name,
                           encoding='utf-8')
    all_data=[] #存储所有的视图数据
    
    for each in view_name:
        
        data= engine.execute("SELECT * FROM %s"%each).fetchall()
        columns_name = engine.execute("SHOW FULL COLUMNS FROM %s"%each)                                  
        columns_name=list(columns_name.fetchall())  # 读取原视图的列名并转换为list后赋给变量
        
        name=[]
        for i in columns_name:
            name.append(list(i)[0])
        data_df=pd.DataFrame(data,columns=name) # 将读取结果转为pd.DataFrame格式,并设定columns
        
        all_data.append(data_df)
    all_data_df = pd.concat(all_data,axis=1)#横向合并
    date = all_data_df.iloc[:,0] #保留日期列
    all_data_df.drop([dropdate],axis=1,inplace=True) #删除的日期重复列
    all_data_df['date'] = date #加上日期列
    return all_data_df


def data_by_period(data,start_time):
    """
    :func:根据起始时间对数据进行截取
    :param: start_year: 起始年份
    :param: data: 固定参数，写入数据
    :return: data_df:数据dataframe
    """ 
    str_p = start_time #起始日期
    date_p = datetime.datetime.strptime(str_p,'%Y-%m-%d').date()
    s = data[data['date']==date_p].index.tolist() #获取起始日期的索引
    data_df = data.iloc[s[0]:]
    data_df.reset_index(inplace=True)
    return data_df


def rates_pct(data):
        """
        :func:对利差/利率进行排列，获取每一个时点利率对应的历史迄今百分比，百分比越小 利差/利率在历史数据中所处水平就越低
        :param: start_year: 起始年份
        :param: data: 固定参数，写入数据
        :return: percentile:百分比表
        """ 
        col_name = [column for column in data]
        rates_pct = pd.DataFrame()
        rates_pct['date'] = data['date']
        
        for j in range(data.shape[1]-1): #跳过最后一列date            
            rank= [0 for j in range(len(data))] #初始值为0的定长list，存储排序结果
            for i in range(1,len(data)): 
                sequence = [] 
                sequence = data.iloc[:i,j].rank() #空值不影响排序
                rank[i-1] = round((sequence[i-1]/i)*100,4) #百分比保留两位小数
            
            rates_pct['%s_pct'%col_name[j]] = rank
                    
        return rates_pct
    

#def get_real_name(col_name,dict_rates):
    #real_name = []
    #for each in col_name:
        #real_name = dict_rates[dict_rates['id']==each]['eng_name']
    #return real_name


def insert_db(df, view_name):
    """
       :func:将dataframe以指定viewname导入sql的特定数据库中（数据库需存在，不能创建）
       :param: df 固定参数，写入的df数据
       :param: view_name: 固定参数，需写入的视图
    """
    #在执行to_sql方法时，将映射好列名和指定类型的dict赋值给dtype参数，映射函数如下
    dtypedict = mapping_df_types(df)
    engine = create_engine('mysql+pymysql://'+user+":"+password+"@"+db_ip+":"+str(port)+"/"+db_name, encoding='utf-8')

    try:
        df.to_sql(view_name, con=engine, index=False, dtype=dtypedict, if_exists='append')
        #if_exits=append：若表存在，将数据写到原表的后面
        print("update successful")
    except:
        print("Error to create view,see the detail")

def mapping_df_types(df):
    """
       :func:将pandas.DataFrame中列名和预指定的类型映射起来
       :param: df固定参数，写入的df数据
       :return: 返回dtypedict
    """    
    dtypedict = {}
    for i, j in zip(df.columns, df.dtypes):
        if "float" in str(j):
            dtypedict.update({i: Float(precision=2, asdecimal=True)})
        if "int" in str(j):
            dtypedict.update({i: Integer()})
    return dtypedict

In [7]:
if __name__ == '__main__':
    """
      数据库的全局变量
    """

    #全局变量：数据库设置参数
    user = 'root'
    password = 'Welcome123'
    db_ip = "cdb-3gsotx9q.cd.tencentcdb.com"
    db_name = 'rates'
    port = 10059 
    
    #设置需要获取的利率视图名称列表
    view_name = ['termsprd_cgb_sdb','termsprd_cgb_sdb_1y','termsprd_cgb_sdb_3y','termsprd_cgb_sdb_5y','termsprd_cgb_sdb_7y','termsprd_cgb_sdb_on']    
    #获取利差数据，并进行处理
    data = get_data_from_view(user,password,db_ip,db_name,port,view_name,'DATE')    
    #计算利差百分比：以2002、2005、2011为起始年,并写入数据库
    df_2002 = rates_pct(data)
    insert_db(df_2002,'termsprd_cgb_sdb_pct_since2002')
    
    data_2005=data_by_period(data,'2005-01-04')
    df_2005 = rates_pct(data_2005)
    insert_db(df_2005,'termsprd_cgb_sdb_pct_since2005')
    
    data_2011=data_by_period(data,'2011-01-04')
    df_2011 = rates_pct(data_2011)
    insert_db(df_2011,'termsprd_cgb_sdb_pct_since2011')

    
    #获取利率数据，并进行处理
    table_name = ['CGB_eng','SDB_eng','ADB_eng','EXIM_eng']
    data_rate = get_data_from_view(user,password,db_ip,db_name,port,table_name,'date')
    #计算利率百分比：以2002、2005、2011为起始年,并写入数据库
    df_rate2002 = rates_pct(data_rate)
    insert_db(df_rate2002,'typical_rates_key_terms_pct_since2002')
    
    data_rate2005=data_by_period(data_rate,'2005-01-04')
    df_rate2005 = rates_pct(data_rate2005)
    insert_db(df_rate2005,'typical_rates_key_terms_pct_since2005')
    
    data_rate2011=data_by_period(data_rate,'2011-01-04')
    df_rate2011 = rates_pct(data_rate2011)
    insert_db(df_rate2011,'typical_rates_key_terms_pct_since2011')
    
    

update successful
update successful
update successful
update successful
update successful
update successful
